In [7]:
import pandas as pd
from textblob import TextBlob
import nltk
import numpy as np
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,Conv1D,MaxPooling1D,GlobalMaxPooling1D,Flatten,SimpleRNN
from keras import regularizers
from sklearn.model_selection import train_test_split 
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
from keras.layers import Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed 
from keras import Input

In [11]:
from sklearn.preprocessing import MinMaxScaler

In [2]:
c
len(df)

232074

In [3]:
df.head(10)

,Unnamed: 0,text,class
0,0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,1,Am I weird I don t get affected by compliments...,non-suicide
2,2,Finally is almost over So I can never hear...,non-suicide
3,3,i need helpjust help me im crying so hard,suicide
4,4,I m so lostHello my name is Adam and I ve b...,suicide
5,5,Honetly idkI dont know what im even doing here...,suicide
6,6,Trigger warning Excuse for self inflicted bu...,suicide
7,7,It ends tonight I can t do it anymore I quit,suicide
8,8,Everyone wants to be edgy and it s making me...,non-suicide
9,9,My life is over at years oldHello all I am a...,suicide


In [6]:
l=[]
for i in range(len(df)):
    l.append(TextBlob(df.iloc[i,1]).sentiment.polarity)
l1=[]
for i in l:
    if i>=0.2:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[:,2].tolist()


In [7]:
from sklearn.metrics import accuracy_score
print(accuracy_score(l1,l2))

0.5741358359833502


In [89]:
#nltk.download('wordnet')
#nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\metul\AppData\Roaming\nltk_data...


True

In [8]:
df=df.iloc[0:100,:]
len(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   text        100 non-null    object
 2   class       100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [11]:

def cleaning(df):
    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Lemmatization
    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df
#Generating Embeddings using tokenizer
data_cleaned=cleaning(df)
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['text'].values)
X = pad_sequences(X)
#Model Building
model = Sequential()
'''
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))'''
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(150, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(X,df["class"],test_size=0.2)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
#Model Training
print(X_train,X_test,y_train,y_test)
model.fit(X_train, y_train, epochs = 20, batch_size=32, verbose =1)
#Model Testing
model.evaluate(X_test,y_test)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 1568, 120)         60000     
                                                                 
 spatial_dropout1d_8 (Spatia  (None, 1568, 120)        0         
 lDropout1D)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 300)               505200    
                                                                 
 dense_16 (Dense)            (None, 150)               45150     
                                                                 
 dense_17 (Dense)            (None, 3)                 453       
                                                                 
Total params: 610,803
Trainable params: 610,803
Non-trainable params: 0
________________________________________________

[0.6433237195014954, 0.6000000238418579]

In [101]:
print(X_train,len(X_train))
print(X_test,len(X_test))
print(y_train,len(y_train))
print(y_test,len(y_test))

[[  0   0   0 ...  20   7   3]
 [  0   0   0 ...  73 229 113]
 [  0   0   0 ...   2  15   6]
 ...
 [  0   0   0 ...  83  10  11]
 [  0   0   0 ... 258   2  66]
 [  0   0   0 ...  54  58  33]] 80
[[  0   0   0 ...  21   5 282]
 [  0   0   0 ...   5   8   6]
 [  0   0   0 ...  28  48   7]
 ...
 [  0   0   0 ...  52   2 333]
 [  0   0   0 ...  26   9  21]
 [  0   0   0 ...   3   7  81]] 20
49    non-suicide
25        suicide
48        suicide
86        suicide
15    non-suicide
         ...     
87        suicide
66    non-suicide
19        suicide
29    non-suicide
9         suicide
Name: class, Length: 80, dtype: object 80
99        suicide
85        suicide
93        suicide
92    non-suicide
5         suicide
62        suicide
72    non-suicide
80    non-suicide
55    non-suicide
39    non-suicide
73        suicide
35    non-suicide
24    non-suicide
23    non-suicide
56    non-suicide
68    non-suicide
20        suicide
16        suicide
52    non-suicide
63    non-suicide
Name: clas

In [103]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
l1=sentiment_pipeline(df.iloc[0:100].tolist())


l1=[]
for i in l:
    if i>=0:
        l1.append("non-suicide")
    else:
        l1.append("suicide")
l2=df.iloc[0:100,2].tolist()
print(accuracy_score(l1,l2))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

d:\tools2\anaconda\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\metul\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

AttributeError: 'DataFrame' object has no attribute 'tolist'

SECOND IMPLEMENTATION


In [2]:
df = pd.read_csv('/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv')

In [4]:
df=pd.read_csv("/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed2.csv")
#df = pd.read_csv( "/Users/aman/coding stuff/sem 6/tarp/dataset/Suicide_preprocessed1.csv", index_col=None, header=0, engine='python' )
df.head()

ParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [3]:
def cleaning(df):
    df['text'] = df['text'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
    # Lemmatization
    df['text'] = df['text'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df
#Generating Embeddings using tokenizer
data_cleaned=cleaning(df)
tokenizer = Tokenizer(num_words=500, split=' ') 
tokenizer.fit_on_texts(data_cleaned['text'].values)
X = tokenizer.texts_to_sequences(data_cleaned['text'].values)
X = pad_sequences(X)

KeyboardInterrupt: 

In [5]:
#Model Building
model = Sequential()
'''
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(704, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(352, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))'''
'''
model.add(Embedding(50, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model.add(MaxPooling1D(5))
model.add(Conv1D(20, 6, activation='relu',kernel_regularizer=regularizers.l1_l2(l1=2e-3, l2=2e-3),bias_regularizer=regularizers.l2(2e-3)))
model.add(GlobalMaxPooling1D())
model.add(LSTM(30, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(15, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])'''

model.add(Embedding(50, 100, input_length=X.shape[1]))
#model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
#model.add(MaxPooling1D(pool_size=2))
model.add(SimpleRNN(10, return_sequences=True))
model.add(LSTM(10, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(15, activation='LeakyReLU'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
df["class"]=preprocessing.LabelEncoder().fit_transform(df["class"])
X_train,X_test,y_train,y_test=train_test_split(X,df["class"],test_size=0.2)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)
#Model Training
print(X_train,X_test,y_train,y_test)
model.fit(X_train, y_train, epochs = 2, batch_size=32, verbose =1)
#Model Testing
model.evaluate(X_test,y_test)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-08 21:45:07.497530: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-08 21:45:07.497560: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 9685, 100)         5000      
                                                                 
 simple_rnn (SimpleRNN)      (None, 9685, 10)          1110      
                                                                 
 lstm (LSTM)                 (None, 10)                840       
                                                                 
 dense (Dense)               (None, 15)                165       
                                                                 
 dense_1 (Dense)             (None, 3)                 48        
                                                                 
Total params: 7,163
Trainable params: 7,163
Non-trainable params: 0
_________________________________________________________________
None
[[  0   0   0 ...  32   2 122]
 [  0   0   0 ... 

2023-02-08 21:45:12.216530: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-08 21:45:13.608191: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [15]:
def shaping(datain, timestep, scaler):
    
    # Convert input dataframe to array and flatten
    arr=datain.to_numpy().flatten()
    
    # Scale 
    arr_scaled=scaler.fit_transform(arr.reshape(-1, 1)).flatten()
    
    cnt=0
    for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range 
        cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
        X_start=mth # Start month for inputs of each sample
        X_end=mth+timestep # End month for inputs of each sample
        Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
        Y_end=mth+2*timestep # End month for targets of each sample.  
        
        # Assemble input and target arrays containing all samples
        if mth==0:
            X_comb=arr_scaled[X_start:X_end]
            Y_comb=arr_scaled[Y_start:Y_end]
        else: 
            X_comb=np.append(X_comb, arr_scaled[X_start:X_end])
            Y_comb=np.append(Y_comb, arr_scaled[Y_start:Y_end])
    
    # Reshape input and target arrays arrays 
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [18]:
##### Step 1 - Specify parameters
timestep=18
location='Canberra'
scaler = MinMaxScaler([-1,0,1])

##### Step 2 - Prepare data

# Split data into train and test dataframes
df_train=df.iloc[:, 0:-2*timestep].copy()
df_test=df.iloc[:, -2*timestep:].copy()

# Select one location
dfloc_train = df_train[df_train.index==location].copy()
dfloc_test = df_test[df_test.index==location].copy()

# Use previously defined shaping function to reshape the data for LSTM
X_train, Y_train = shaping(datain=dfloc_train, timestep=timestep, scaler=scaler)
X_test, Y_test = shaping(datain=dfloc_test, timestep=timestep, scaler=scaler)


##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="LSTM-Model") # Model
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False), name='Hidden-LSTM-Encoder-Layer')) # Encoder Layer
model.add(RepeatVector(Y_train.shape[1], name='Repeat-Vector-Layer')) # Repeat Vector
model.add(Bidirectional(LSTM(units=32, activation='tanh', recurrent_activation='sigmoid', stateful=False, return_sequences=True), name='Hidden-LSTM-Decoder-Layer')) # Decoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x

##### Step 4 - Compile the model
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=1, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=20, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=0, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=100, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                   )







ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
##### Step 6 - Use model to make predictions
# Predict results on training data
pred_train = model.predict(X_train)
# Predict esults on test data
pred_test = model.predict(X_test)

In [ ]:
##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")